Implementing Monaco examples in Lea
===================================

Introduction
------------

The Monaco program uses Monte Carlo simulation to calculate probability distributions for games. It uses its own custom language for defining the probability to be simulated.

Monaco offers both simulation-based results and (for sufficiently simple cases) exact calculation of probabilities. It is coded in C++ for speed, and has a wealth of outputoptions.

The Python library [lea](https://bitbucket.org/piedenis/lea) implements discrete probability distributions in Python. It calculates exact distributions based on calculations built up from primitive distribution objects.

The Examples
------------

The Monaco documentation comes with a number of example calculations, ranging from very simple (rolling dice to generate D&D characters) to extremely complex (a simulation of Cribbage). In this notebook, we are going to attempt to reproduce the Monaco examples using Lea, as an exercise to compare the power of the two tools.

**Disclaimer** The Monaco program has been developed over an extensive period and is written specifically for the task in hand in a compiled language. On the other hand, I have had little or no prior experience with lea, and the code runs in interpreted Python. Furthermore, apart from the Monaco "exact" mode, the two programs produce their results in completely different ways. While I will be looking at performance of the various calculations, this is simply for interest and not in any way a comparison of the two programs.

In [1]:
from lea import *

We're also going to want to work out how long it takes to get our results. This is for two main reasons. First, as a bit of amusement to see how well our code compares with Monaco. As noted above, Monaco is customised for this task, and has a lot of options that we're not even looking at here, so "which approach is faster" comparisons are somewhat pointless. But I find it interesting to know how competitive my "use a high-level language and find a library that's already written to do the hard stuff" approach is in practice. And secondly, the naive approach to some of the following problems scales *really* badly, so we'll time our code to get an idea of when things start to go bad and we need to do some optimisation.

In [2]:
import time
from contextlib import contextmanager
from datetime import timedelta

@contextmanager
def time_this(title="Execution time"):
    start = time.perf_counter()
    yield
    end = time.perf_counter()
    print("{}: {}".format(title, timedelta(seconds=end-start)))
    
# A quick test
with time_this():
    time.sleep(0.5)

Execution time: 0:00:00.500644


Example 1 - D&D Characters
--------------------------

The first example is rolling the characteristics for a D&D character. The process is to roll five six-sided dice, discard the worst two, and add the rest up. In Monaco, this is

    sum(tail3(sort(repeat5{d6})))

In lea, we will separate those steps out.

In [3]:
# First, the probability distribution for a single six-sided die
d6 = Lea.interval(1,6)
# We now take the cartesian product of five of these
# This gives us a distribution of tuples, each outcome containing a specific set of five dice rolls
five_d6 = d6.cprodTimes(5)

# The following function, given a set of die rolls, calculates the characteristic based on them
def characteristic(dice):
    return sum(list(sorted(dice, reverse=True))[:3])

# So we map the calculation over the dieroll distribution, and display the results
with time_this("Map"):
    distribution = five_d6.map(characteristic)

# Display as a histogram
with time_this("Display"):
    print(distribution.asString('/-'))

Map: 0:00:00.000012
 3 :    1/7776 
 4 :    5/7776 
 5 :   15/7776 
 6 :   41/7776 -
 7 :   90/7776 -
 8 :  170/7776 --
 9 :  296/7776 ----
10 :  470/7776 ------
11 :  665/7776 ---------
12 :  881/7776 -----------
13 : 1055/7776 --------------
14 : 1155/7776 ---------------
15 : 1111/7776 --------------
16 :  935/7776 ------------
17 :  610/7776 --------
18 :  276/7776 ----
Display: 0:00:00.103609


In principle, this approach (generate all the results we need, define a function to use them to calculate the required value, map the function over the results) works for any type of calculation. But there's an obvious issue, which is that the intermediate cartesian product could be very large. Whether that is an issue in practice, is something we'll need to determine as we proceed. The lea documentation states that lea calculates distributions lazily, so it should be possible to limit the combinatorial explosion.

One notable consequence of the lazy evaluation approach is that calculating the distribution is essentially free, but producing the results for display is where the time is spent (only 0.1 seconds here, but we're going to see bigger times soon...)

Example 2 - Monopoly
--------------------

The second example in the Monaco documentation is calculatingthe expected move around a Monopoly board. The rule is a little more complex this time. Roll 2 dice initially, and add the values. If you rolled a double, reroll and add the result to the first roll. Repeat as often as you roll a double, but if you roll three doubles, you "go to gaol" instead of moving.

The Monaco calculation is

    until(r1:=d6;r2:=d6;r0+=r1+r2,r1!=r2|(r9+=1)==3&error)

There are two interesting points here - `until` only calculates as needed, so Monaco doesn't roll extra dice if the first pair wasn't a double. And `error` is used to signal the "go to gaol" condition. Note that because of the use of `until`, Monaco cannot calculate this result exactly - but that's not really important as a simple reformulation to pre-roll all the dice:

    r:=repeat6{d6};r0+r1+(r0==r1?r2+r3+(r2==r3?(r4==r5?error:r4+r5:0):0)

can be calculated exactly.

In lea, we will use the same "do all the rolls and calculate the result" as before. One small point is that because lea doesn't require the result values to all be numbers, we can simply use (the string) "Gaol" as a result for 3 doubles.

In [4]:
# Start as before, this time pregenerate 6 die rolls
six_d6 = Lea.interval(1,6).cprodTimes(6)

# A function that does the actual "given 6 die rolls, what move would
# I make in Monopoly?" calculation.
def monopoly(d):
    d1,d2,d3,d4,d5,d6 = d
    if d1 != d2:
        return d1+d2
    if d3 != d4:
        return d1+d2+d3+d4
    if d5 != d6:
        return d1+d2+d3+d4+d5+d6
    return 'Gaol'

# And as before, map the results and display
distribution = six_d6.map(monopoly)

# Display as a histogram
with time_this("Calculate the distribution"):
    print(distribution.asString('/-'))

   3 : 1296/23328 ------
   4 : 1296/23328 ------
   5 : 2628/23328 -----------
   6 : 2628/23328 -----------
   7 : 3997/23328 -----------------
   8 : 2701/23328 ------------
   9 : 2812/23328 ------------
  10 : 1480/23328 ------
  11 : 1594/23328 -------
  12 :  225/23328 -
  13 :  342/23328 -
  14 :  231/23328 -
  15 :  351/23328 --
  16 :  237/23328 -
  17 :  324/23328 -
  18 :  207/23328 -
  19 :  259/23328 -
  20 :  139/23328 -
  21 :  154/23328 -
  22 :   67/23328 
  23 :   79/23328 
  24 :   27/23328 
  25 :   36/23328 
  26 :   21/23328 
  27 :   27/23328 
  28 :   15/23328 
  29 :   18/23328 
  30 :    9/23328 
  31 :   10/23328 
  32 :    4/23328 
  33 :    4/23328 
  34 :    1/23328 
  35 :    1/23328 
Gaol :  108/23328 
Calculate the distribution: 0:00:00.217309


An interesting (but minor) point is that the display of the results is not in any particular order. In non-experimental code, we'd fix this, but for now the simple display routine is sufficient.

### Digression - Short Circuiting

The overhead of rolling all the dice up front isn't significant here, so the above approach is fine. But it's not going to scale for long (see below). So we need to consider howe we'd implement a version of the calculation that short circuits. This gets us quite a lot further into the details of how lea builds distributions, and how lazy evaluation works.

Here is a short-circuiting implementation of the Monopoly proble, from the author of lea. Note that it doesn't allow for the "Gaol" result, instead simply allowing a maximum number of rerolls.

In [5]:
def recDice(nbMaxThrows):
    d1 = Lea.interval(1,6)
    d2 = d1.clone()
    dSum = d1 + d2
    if nbMaxThrows > 1:
        dSum += Lea.if_(d1==d2,recDice(nbMaxThrows-1),0)
    return dSum.getAlea()

recDice(3)

 3 : 2592/46656
 4 : 2592/46656
 5 : 5256/46656
 6 : 5257/46656
 7 : 7994/46656
 8 : 5405/46656
 9 : 5624/46656
10 : 2966/46656
11 : 3188/46656
12 :  460/46656
13 :  684/46656
14 :  477/46656
15 :  702/46656
16 :  495/46656
17 :  648/46656
18 :  439/46656
19 :  518/46656
20 :  305/46656
21 :  308/46656
22 :  161/46656
23 :  158/46656
24 :   79/46656
25 :   72/46656
26 :   63/46656
27 :   54/46656
28 :   45/46656
29 :   36/46656
30 :   28/46656
31 :   20/46656
32 :   14/46656
33 :    8/46656
34 :    5/46656
35 :    2/46656
36 :    1/46656

The results differ from the Monopoly results because of the lack of "Gaol" - a roll of 1,1,1,1,1,1 counts as 6, rather than as "go to gaol".

The way the above function works is important, though.

Internally, lea uses various different object types for probability distributions. The `Alea` type is the fundamental "results with weights" type, and as such, is fast to use. The `Clea` type is the other one we've been using so far. It represents a cartesian join of distributions, but it doesn't actually calculate any results until needed (at which point it does the calculations and caches the result, which is why the time spent in our examples so far is when we display the results).

The above function uses a new type, `Blea`, which represents a conditional probability. So `Lea.if_(cond, t, f)` builds a structure that chooses `t` or `f` depending on the value of `cond`. We use this to define an object that only includes the 3rd, 4th and subsequent dice if needed. (The code as given then forces the evaluation of the probabilities by explicitly requesting an `Alea` object at the end, but if it didn't, the usual "evaluate on demand" would occur).

Adding in the "Gaol" result is harder to do than it looks. The problem is that we are adding 0 to the sum (rather than a further recursive segment of the expression tree) to terminate the recursion on a non-equal pair, but when we hit the maximum number of rerolls, there's no easy way to push an "exceptional" result back up to the top level. It can be done, but the implementation is tricky. Basically you have to add control-flow into a purely expression-based calculation. Functional programmers are probably wondering what the fuss is all about, but from an imperative perspective, it's hard to get your mind round the necessary contortions.

The following is a working implementation, but it's hard to love it. I'm using an anonymous function (lambda) and immediately calling it, to simulate the use of temporary variables in an expression. There are certainly better ways of doing this. But we'll leave this topic for now, as the principle is established, and we'll likely come back to it later, for more complex problems.

In [6]:
def recMonopolyDice(nbMaxThrows):
    d1 = Lea.interval(1,6)
    d2 = d1.clone()
    dSum = d1 + d2
    if nbMaxThrows == 1:
        dSum = Lea.if_(d1==d2,"Gaol",dSum)
    else:
        dSum = Lea.if_(d1==d2,(lambda x:Lea.if_(x=="Gaol",x,dSum+x))(recMonopolyDice(nbMaxThrows-1)),dSum)
    return dSum.getAlea()   

recMonopolyDice(3)

   3 : 1296/23328
   4 : 1296/23328
   5 : 2628/23328
   6 : 2628/23328
   7 : 3997/23328
   8 : 2701/23328
   9 : 2812/23328
  10 : 1480/23328
  11 : 1594/23328
  12 :  225/23328
  13 :  342/23328
  14 :  231/23328
  15 :  351/23328
  16 :  237/23328
  17 :  324/23328
  18 :  207/23328
  19 :  259/23328
  20 :  139/23328
  21 :  154/23328
  22 :   67/23328
  23 :   79/23328
  24 :   27/23328
  25 :   36/23328
  26 :   21/23328
  27 :   27/23328
  28 :   15/23328
  29 :   18/23328
  30 :    9/23328
  31 :   10/23328
  32 :    4/23328
  33 :    4/23328
  34 :    1/23328
  35 :    1/23328
Gaol :  108/23328

Example 3 - Ysphan
------------------

In many ways, this is more of the same. The first part of the calculation is simply the number of *different* numbers showing on a roll of nine six-sided dice. The second part is the same, but with twelve dice (to look at whether buying 3 extra dice is worth it).

In Monaco

    unique(repeat9{d6})

The new issue here is that we start to get much bigger intermediate cartesian joins. In fact, sufficiently big that running the calculations takes a long time (not just sub-second, but real work, now). So we'll do the nine-dice example, and display the runtime as well.

In [7]:
# The same old routine
nine_d6 = Lea.interval(1,6).cprodTimes(9)

# Number of unique values
def uniq(d):
    return len(set(d))

# Calculate the distribution
distribution = nine_d6.map(uniq)

# Display as a histogram
with time_this("Full cartesian join, 9 dice"):
    print(distribution.asString('/-'))

1 :      1/1679616 
2 :   1275/1679616 
3 :  60500/1679616 ----
4 : 466200/1679616 ----------------------------
5 : 834120/1679616 --------------------------------------------------
6 : 317520/1679616 -------------------
Full cartesian join, 9 dice: 0:01:36.343449


OK, so we learned some interesting things here.

1. The "roll all the dice up front" approach isn't going to scale.
2. Lea does lazy calculations, so the time is in the *display* step, not where you'd immediately expect, in the creation of the distribution.

I'm not going to say how many runs it took me to work out what was going on there. But learning experiences are good :-) I actually went back and reworked some of the earlier presentation so it looks like I knew this earlier. That's the fun of writing things like this up as a document - you can pretend you knew what was going on from the start. But it's also interesting (to me, at least) to record the learning process. And until I hit a case where the calculations were taking a significant time, I hadn't really thought about where the time was spent, so I hadn't thought through the implications of lazy evaluation.

So we want a better approach to this calculation.

Sorted Sets of Dice
-------------------

For the Ysphan example we don't actually need to consider *all* possible rolls - the order we roll the dice in is irrelevant. So we could just as well use a (suitably weighted) probability distribution that covered only the set of rolls that are unique ignoring ordering.

Let's look at generating *sorted* sets of dice. A sorted result is certainly one of the "unique up to ordering" answers, and for some problems having the results sorted might be useful (it would have helped in the D&D example above, for instance). And it turns out not to be any harder to generate sorted results (it's actually easier because there's a standard library function that does it for us :-))

OK, so we're looking for sets of *n* dice, each with *m* sides, such that the dice are in ascending order. The first die can take any value. Then the second can only take values from the value of the first upwards. And so on.

We can generate a set of values by using `itertools.combinations_with_replacement(die, N)` where `die` is the result set of the die, and `N` is how many to take at a time. To get the weights, we need to know how many arrangements of the outcome there are. This is $n!$ (how many permutations of the result there are) divided by the product of the number of orderings of each repeated value (if a value is repeated $r$ times, there are $r!$ such orderings).

Note that `combinations_with_replacement` is defined as emitting the results in lexicographic order, so as long as the input (`range`) is sorted, the output will be too.

In [8]:
from itertools import combinations_with_replacement
from collections import Counter
from math import factorial
def dice_set(n, m):
    permutations = factorial(n)
    for outcome in combinations_with_replacement(range(1, m+1), n):
        weight = permutations
        for v in Counter(outcome).values():
            weight = weight // factorial(v)
        yield outcome, weight

As a test, let's do the D&D example again.

In [9]:
with time_this("D&D with sorted data"):
    print(Lea.fromValFreqs(*dice_set(5,6)).map(lambda d: sum(d[2:])).asString("/-"))

 3 :    1/7776 
 4 :    5/7776 
 5 :   15/7776 
 6 :   41/7776 -
 7 :   90/7776 -
 8 :  170/7776 --
 9 :  296/7776 ----
10 :  470/7776 ------
11 :  665/7776 ---------
12 :  881/7776 -----------
13 : 1055/7776 --------------
14 : 1155/7776 ---------------
15 : 1111/7776 --------------
16 :  935/7776 ------------
17 :  610/7776 --------
18 :  276/7776 ----
D&D with sorted data: 0:00:00.003534


Faster, but for a problem this size it doesn't really matter.

With this routine, let's retry the Ysphan example.

In [10]:
with time_this("Ysphan 9 dice, sorted rolls"):
    rolls = Lea.fromValFreqs(*dice_set(9, 6))
    print(rolls.map(lambda d: len(set(d))).asString('/-'))

1 :      1/1679616 
2 :   1275/1679616 
3 :  60500/1679616 ----
4 : 466200/1679616 ----------------------------
5 : 834120/1679616 --------------------------------------------------
6 : 317520/1679616 -------------------
Ysphan 9 dice, sorted rolls: 0:00:00.025746


Well, that is significantly faster! We can check how much of an improvement there is in terms of the number of outcomes to check. We'll do this for both the 9 and 12 dice cases.

In [11]:
def improvement(n):
    # Cartesian product result set size
    cp = 6**n
    # Order-independent result set size
    oi = len(list(combinations_with_replacement(range(1,7),n)))
    print("For {} dice, the number of outcomes to test drops from {} to {} ({:.4f}% as many)".format(n, cp, oi, oi/cp*100))

improvement(9)
improvement(12)

For 9 dice, the number of outcomes to test drops from 10077696 to 2002 (0.0199% as many)
For 12 dice, the number of outcomes to test drops from 2176782336 to 6188 (0.0003% as many)


 That looks good - and the improvement increases as the number of dice increase :-)
 
 **Note:** I don't know how to calculate the order-independent result set size as a function of the number of results, n. At some point I will work this out so I can include a proper complexity calculation for the `dice_set` function.
 
 Let's see how the 12-dice example fares.

In [12]:
with time_this("Ysphan 12 dice, sorted rolls"):
    rolls = Lea.fromValFreqs(*dice_set(12, 6))
    print(rolls.map(lambda d: len(set(d))).asString('/-'))

1 :         1/362797056 
2 :     10235/362797056 
3 :   1730520/362797056 
4 :  36690060/362797056 ----------
5 : 165528000/362797056 ----------------------------------------------
6 : 158838240/362797056 --------------------------------------------
Ysphan 12 dice, sorted rolls: 0:00:00.094105


OK, so as long as order isn't important we win significantly.

## Example 4 - To Court The King

This example is harder to formulate than the previous ones. The rules are explained in a very procedural manner (hardly surprising, that's how the game rules are written).

The following is the procedural algorithm we'll use.

1. Roll N dice. Put to one side the largest set of dice with the same number on as you can.
2. Roll the remaining dice again, and keep any dice with your chosen number. If none have your chosen number, discard one die.
3. Repeat step (2) as long as you have dice to roll.

In order to produce a probability distribution, we need to express this in terms of an expression.

**Note** This reformulation will be hard. I'll save this document now and come back to this problem later...